In [1]:
import re
import numpy as np
import pandas as pd
from gensim.models import Word2Vec

from underthesea import word_tokenize

from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

import warnings
warnings.filterwarnings("ignore")

# Tiền xử lý dữ liệu

In [2]:
# load data
df = pd.read_csv('chatbot-data.csv', usecols=[1,2])
df

,user_a,user_b
0,Thích mẫu người nào,"Dễ thương, tóc dài, da trắng"
1,Có crush ai không,Có 1 bạn cùng lớp
2,Tại sao lại thích bạn dó,Vì đáp ứng những yêu cầu của tao
3,Có hay nói chuyện không,Hay nhắn tin messenger
4,Bạn kia có bắt chuyện trước không,Có đôi khi
...,...,...
7389,Uầy vậy anh em mình có nên đi thể dục thể thao...,Vậy mày định rủ tao đi tập gì.
7390,Tao thấy bộ môn đó khá được ưa chuộng và còn n...,Mày xem các dân tổ đua xe nhiều quá nên ảo rồi...
7391,Đùa tí thôi tao tính rủ mày đi học võ bro.,Được nè. Tao khá thích taekwondo nhe.
7392,Tao sẽ học lên đai đen để về sút vào mấy thằng...,Oke mày là nhất :))


In [3]:
# lấy ra Questions và Answers
questions = df['user_a'].values
answers = df['user_b'].values

questions[:2], answers[:2]

(array(['Thích mẫu người nào', 'Có crush ai không'], dtype=object),
 array(['Dễ thương, tóc dài, da trắng', 'Có 1 bạn cùng lớp'], dtype=object))

In [4]:
# hàm để xóa các ký tự đặc biệt
def clean_text(sent):
    return re.sub(r'[!“”"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]', '', sent)

# hàm để chuyển word segmentation và chuyển question thành các token
def text_to_tokens(sent):
    return word_tokenize(clean_text(sent.lower()), format='text').split()

In [5]:
# tách Questions thành các token
question_tokens = [text_to_tokens(ques) for ques in questions]
question_tokens[:10]

[['thích', 'mẫu', 'người', 'nào'],
 ['có', 'crush', 'ai', 'không'],
 ['tại_sao', 'lại', 'thích', 'bạn', 'dó'],
 ['có', 'hay', 'nói_chuyện', 'không'],
 ['bạn', 'kia', 'có', 'bắt_chuyện', 'trước', 'không'],
 ['có', 'định', 'rủ', 'bạn', 'đó', 'về', 'quê', 'dịp', 'tết', 'không'],
 ['có', 'hay', 'đi', 'chơi', 'với', 'nhau', 'không'],
 ['sở_thích', 'mày', 'sao'],
 ['thích', 'phim', 'gì'],
 ['thích', 'chơi', 'gì']]

# 1.a Sử dụng độ đo Bleu score

In [6]:
def answer_with_BleuScore(question):
    bleu_score = [sentence_bleu([ques], text_to_tokens(question), weights=(0.5, 0.5)) for ques in question_tokens]
    predict = np.argmax(bleu_score)
    return answers[predict]

In [13]:
def chatbot_BleuScore():
    print('bot: Xin chào!')
    while True:
        question = input('me: ')
        if question == '':
            print('bot: Gud Bie')
            break
        else: 
            print('User: ', question)
        print('bot: ',answer_with_BleuScore(question),'\n')

In [17]:
chatbot_BleuScore()

bot: Xin chào!
User:  Bạn tên gì
bot:  Nam 

User:  bạn học trường nào
bot:  Tớ học đại học Tôn Đức Thắng 

User:  đang làm sinh viên năm mấy vậy
bot:  Mình sinh viên năm 3 

User:  có được học bổng không
bot:  2 cái 

User:  có học bổng không
bot:  có em 

User:  bạn thích gì nhất
bot:  Đủ thứ hết 

User:  khi rảnh bạn làm gì
bot:  chơi game và nghe nhạc 

bot: Gud Bie


# 1.b Sử dụng Word2Vec

In [18]:
len(question_tokens)

7394

In [19]:
vector_size = 150
modelw2v = Word2Vec(question_tokens, vector_size=vector_size, window=10, sg=1, workers=4, min_count=2)

def convert2vec(words):
    sum = np.zeros((1, vector_size), dtype=int)[0]
    for word in words:
        if not (word in modelw2v.wv):
            continue
        vec = modelw2v.wv[word]
        sum = sum + vec
    return sum/len(words) # Vector của câu size (1,vector_size)
X = [convert2vec(ques) for ques in question_tokens]

### Độ đo Euclid

In [20]:
def answer_with_w2v_Euclid(question):
    input_ques = convert2vec(text_to_tokens(question))
    q = euclidean_distances([input_ques], X)
    predict = np.argmin(q,axis=1)
    return answers[predict[0]]

In [22]:
def chatbot_w2v_Euclid():
    print('bot: Xin chào!')
    while True:
        question = input('me: ')
        if question == '':
            print('bot: bye')
            break
        else: 
            print('User: ', question)
        print('bot: ',answer_with_w2v_Euclid(question),'\n')

In [23]:
chatbot_w2v_Euclid()

bot: Xin chào!
User:  xin chào bạn tên gì?
bot:  Tôi tên Hưng. Chào bạn? 

User:  bạn học trường nào
bot:  Tớ học đại học Tôn Đức Thắng 

User:  bạn có sở thích gì không
bot:  Đọc truyện 

User:  bạn có đi chơi đâu không
bot:  đi biển 

User:  Bạn có rảnh không
bot:  Mình thường đi cùng bạn ấy và nhóm bạn đại học. 

User:  tạm biệt
bot:  Tạm biệt 

bot: bye


### Độ đo Cosine

In [24]:
def answer_with_w2v_Cosine(question):
    input_ques = convert2vec(text_to_tokens(question))
    q = cosine_similarity([input_ques], X)
    predict = np.argmax(q,axis=1)
    return answers[predict[0]]

In [25]:
def chatbot_w2v_Cosine():
    print('bot: Xin chào!')
    while True:
        question = input('me: ')
        if question == '':
            print('bot: bye')
            break
        else: 
            print('User: ', question)
        print('bot: ',answer_with_w2v_Cosine(question),'\n')

In [28]:
chatbot_w2v_Cosine()

bot: Xin chào!
User:  bạn tên gì
bot:  Nam 

User:  học trường nào vậy
bot:  Trường đại học Buôn Ma Thuột 

User:  bạn học ngành gì
bot:  mình học Công Nghệ Thông Tin 

User:  học năm mấy rồi
bot:  đại học năm 3 

User:  ngành đó khó không
bot:  Khá là khó 

bot: bye
